**主要内容：**
- 超参数调整
- BN
- softmax
- tensorflow框架

# 超参数调试
* 机器学习领域，参数较少，采用网格方式调整
* dl领域，参数较多时候，采用随机选择点进行，在某一区域找到效果好的点后在继续在该范围内寻找

* Scale均匀随机选择
一般的，如果在$10^a∼10^b$之间的范围内进行按比例的选择，则$r \in [a, b]，α=10^r$。同样，在使用指数加权平均的时候，超参数β也需要用上面这种方向进行选择.
```
r = -4 * np.random.rand() # r in [-4,0]
learning_rate = 10 ** r # 10^{r}
```
* 超参数调整方法：pandas vs caviar
   - 资源有限时候，一次调整一个参数
   - 资源充足时候，并行调试多个模型

# 网络中激活值得归一化
在Logistic Regression 中，将输入特征进行归一化，可以加速模型的训练。那么对于更深层次的神经网络，我们是否可以归一化隐藏层的输出$a^{[l]}$或者经过激活函数前的$z^{[l]}$，以便加速神经网络的训练过程？答案是肯定的。

**常用的方式是将隐藏层的经过激活函数前的$z^{[l]}$进行归一化。**


## BN的实现
以神经网络中某一隐藏层的中间值为例：$z^{(1)},z^{(2)},\ldots,z^{(m)}$
$$\mu = \dfrac{1}{m}\sum\limits_{i}z^{(i)}
\\ \sigma^{2}=\dfrac{1}{m}\sum\limits_{i}(z^{(i)}-\mu)^{2}
\\z^{(i)}_{\rm norm} = \dfrac{z^{(i)}-\mu}{\sqrt{\sigma^{2}+\varepsilon}}$$
这里加上ε是为了保证数值的稳定。到这里所有z的分量都是平均值为0和方差为1的分布，但是我们不希望隐藏层的单元总是如此，也许不同的分布会更有意义，所以我们再进行计算：$$\widetilde z^{(i)} = \gamma z^{(i)}_{\rm norm}+\beta$$
这里γ和β是可以更新学习的参数，如神经网络的权重w一样，两个参数的值来确定z˜(i)所属的分布。

## NN中的BN
![BN](./Image/BN.png)
反向传播时候还需要就是那BN参数的梯度。

**注意**

无论$b^{[l]}$值为多少，在这个过程中都会被减去，不会再起作用。所以如果在神经网络中应用Batch Norm 的话，就直接将偏置参数$b^{[l]}$去掉，或者将其置零。

## Batch Norm 起作用的原因
* First Reason

首先Batch Norm 可以加速神经网络训练的原因和输入层的输入特征进行归一化，从而改变Cost function的形状，使得每一次梯度下降都可以更快的接近函数的最小值点，从而加速模型训练过程的原理是有相同的道理。
只是Batch Norm 不是单纯的将输入的特征进行归一化，而是将各个隐藏层的激活函数的激活值进行的归一化，并调整到另外的分布。
* Second Reason

对于后面的神经网络，是以第二层隐层的输出值a[2]作为输入特征的，通过前向传播得到最终的y^，但是因为我们的网络还有前面两层，由于训练过程，参数w[1]，w[2]是不断变化的，那么也就是说对于后面的网络，a[2]的值也是处于不断变化之中，所以就有了Covariate shift的问题。
那么如果对z[2]使用了Batch Norm，那么即使其值不断的变化，但是其均值和方差却会保持。那么Batch Norm的作用便是其限制了前层的参数更新导致对后面网络数值分布程度的影响，使得输入后层的数值变得更加稳定。另一个角度就是可以看作，Batch Norm 削弱了前层参数与后层参数之间的联系，使得网络的每层都可以自己进行学习，相对其他层有一定的独立性，这会有助于加速整个网络的学习。


## Batch Norm 正则化效果
Batch Norm还有轻微的正则化效果。

这是因为在使用Mini-batch梯度下降的时候，每次计算均值和偏差都是在一个Mini-batch上进行计算，而不是在整个数据样集上。这样就在均值和偏差上带来一些比较小的噪声。那么用均值和偏差计算得到的z˜[l]也将会加入一定的噪声。

## 在测试数据上使用 Batch Norm
训练过程中，我们是在每个Mini-batch使用Batch Norm，来计算所需要的均值μμ和方差σ2σ2。但是在测试的时候，我们需要对每一个测试样本进行预测，无法计算均值和方差。

此时，我们需要单独进行估算均值μ和方差σ^2。通常的方法就是在我们训练的过程中，对于训练集的Mini-batch，使用指数加权平均，当训练结束的时候，得到指数加权平均后的均值μ和方差σ^2，而这些值直接用于Batch Norm公式的计算，用以对测试样本进行预测。


# Softmax 回归
在多分类问题中，有一种 logistic regression的一般形式，叫做Softmax regression。Softmax回归可以将多分类任务的输出转换为各个类别可能的概率，从而将最大的概率值所对应的类别作为输入样本的输出类别。

## 理解 Sotfmax
![softmax](./Image/softmax.png)

$$L(\hat y,y)=-\sum\limits_{j=1}^{4}y_{j}\log \hat y_{j}$$
$$J(w^{[1]},b^{[1]},\ldots)=\dfrac{1}{m}\sum\limits_{i=1}^{m}L(\hat y^{(i)},y^{(i)})$$
$$\dfrac{\partial J}{\partial z^{[L]}}=dz^{[L]} = \hat y -y$$